In [18]:
import gradio as gr
import pandas as pd
import os
from deep_translator import GoogleTranslator
from langdetect import detect

In [19]:
lang_map = {
    "English": "en",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Russian": "ru",
    "Hindi": "hi",
    "Bengali": "bn",
    "Armenian": "hy",
    "Icelandic": "is",
    "Esperanto": "eo"
}

In [20]:
FLAGGED_FILE = "flagged_translations.csv"

def translate_text(from_text, target_lang):
    try:
        if target_lang not in lang_map:
            return "Error: Invalid target language"

        # ✅ Correctly Detect the Source Language
        detected_lang = detect(from_text)

        # ✅ Translate Text
        translated = GoogleTranslator(source=detected_lang, target=lang_map[target_lang]).translate(from_text)

        return f"(Detected: {detected_lang.upper()}) {translated}"
    
    except Exception as e:
        return f"Error: {str(e)}"


# ✅ Improved Flagging Function
def flag_translation(input_text, translation, target_lang, reason):
    try:
        detected_lang = detect(input_text)  # Extract detected language
        print(f"🚩 Flagging Detected Lang: {detected_lang}")  # Debugging Print

        new_entry = pd.DataFrame([{
            "Input Text": input_text,
            "Detected Language": detected_lang,
            "Target Language": target_lang,
            "Translation": translation,
            "Reason": reason
        }])

        # ✅ Ensure file exists
        if not os.path.exists(FLAGGED_FILE):
            new_entry.to_csv(FLAGGED_FILE, index=False, encoding="utf-8")
        else:
            df = pd.read_csv(FLAGGED_FILE, encoding="utf-8")
            df = pd.concat([df, new_entry], ignore_index=True)
            df.to_csv(FLAGGED_FILE, index=False, encoding="utf-8")
        
        return "✅ Translation flagged successfully!"
    except Exception as e:
        return f"❌ Flagging Error: {str(e)}"

In [21]:
# ✅ Gradio Interface
with gr.Blocks() as interface:
    gr.Markdown("### 🌍 Language Translator with Flagging")
    
    input_text = gr.Textbox(label="Enter Text")
    target_lang = gr.Dropdown(list(lang_map.keys()), label="Target Language")
    output_text = gr.Textbox(label="Translated Text")

    translate_btn = gr.Button("Translate")
    translate_btn.click(translate_text, inputs=[input_text, target_lang], outputs=output_text)

    gr.Markdown("### 🚩 Flag Translation")
    flag_reason = gr.Dropdown(["Incorrect Translation", "Offensive Content", "Other"], label="Reason")
    flag_btn = gr.Button("Flag Translation")
    flag_msg = gr.Textbox(label="Flagging Status", interactive=False)

    flag_btn.click(flag_translation, inputs=[input_text, output_text, target_lang, flag_reason], outputs=flag_msg)

In [22]:
interface.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
